In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [106]:
#df_1 parecería no tener problemas con sus caracteres, df_2 si. 
df_1 = pd.read_csv('cine_nacional_trabajada.csv', error_bad_lines=False, encoding='utf-8')
df_2 = pd.read_csv('cine_nacional_trabajada_aj_man.csv', encoding= 'latin1', error_bad_lines=False)

Skipping line 3: expected 4 fields, saw 6
Skipping line 15: expected 4 fields, saw 5
Skipping line 22: expected 4 fields, saw 5
Skipping line 43: expected 4 fields, saw 5
Skipping line 58: expected 4 fields, saw 6
Skipping line 113: expected 4 fields, saw 5
Skipping line 118: expected 4 fields, saw 6
Skipping line 279: expected 4 fields, saw 5
Skipping line 330: expected 4 fields, saw 6
Skipping line 402: expected 4 fields, saw 5
Skipping line 472: expected 4 fields, saw 5
Skipping line 478: expected 4 fields, saw 7
Skipping line 518: expected 4 fields, saw 5
Skipping line 559: expected 4 fields, saw 5
Skipping line 599: expected 4 fields, saw 7
Skipping line 639: expected 4 fields, saw 12
Skipping line 680: expected 4 fields, saw 5
Skipping line 681: expected 4 fields, saw 7
Skipping line 727: expected 4 fields, saw 6
Skipping line 743: expected 4 fields, saw 8
Skipping line 770: expected 4 fields, saw 6
Skipping line 833: expected 4 fields, saw 7
Skipping line 846: expected 4 fields,

In [107]:
#Sacamos 18 datos sin sinopsis
df_1.columns = ['ID', 'Titulo','Anio de Produccion', 'Sinopsis']

df_1 = df_1[df_1['Sinopsis'].notnull()]

In [117]:
#Sacamos <br> <i> y todo bracket html que haya quedado en el texto.

import re

def remove_html_signs(n):
    return re.sub(r'\<[^>]*\>','',n)

df_1['Sinopsis'].apply(remove_html_signs)

0      En una academia de tango, un hombre melancólic...
1      La estructura familiar de un correcto hombre c...
2      Un muchacho de barrio se separa de su esposa y...
3      Película documental basada en el recital que d...
4      Rey se encuentra con Isabel después de 15 años...
5      Un sacerdote muy particular predica la caridad...
6      Una pareja de estafadores roba un hotel en Mia...
7      Un grupo de estudiantes es sometido por unos c...
8      Nueva mirada del cine argentino sobre la dicta...
9      Un militante político de la década del 70 huye...
10     En el amor es una mujer gorda, Alejandro Agres...
11     Un hombre en bancarrota, acosado por deudas de...
12     Una familia debe sobornar a un empleado de una...
13     La película narra el caso real del senador Lis...
14     Un hombre casado se siente atraído por una sob...
15     Si bien el film no alude a nadie con nombres p...
16     Cuenta la historia de Pikín, un pequeño y simp...
17     Después de diez años de 

In [78]:
#creamos lista de stop-words en español
spanish = {
	"a":               "",
	"acuerdo":         "",
	"adelante":        "",
	"ademas":          "",
	"además":          "",
	"adrede":          "",
	"ahi":             "",
	"ahí":             "",
	"ahora":           "",
	"al":              "",
	"alli":            "",
	"allí":            "",
	"alrededor":       "",
	"antano":          "",
	"antaño":          "",
	"ante":            "",
	"antes":           "",
	"apenas":          "",
	"aproximadamente": "",
	"aquel":           "",
	"aquél":           "",
	"aquella":         "",
	"aquélla":         "",
	"aquellas":        "",
	"aquéllas":        "",
	"aquello":         "",
	"aquellos":        "",
	"aquéllos":        "",
	"aqui":            "",
	"aquí":            "",
	"arribaabajo":     "",
	"asi":             "",
	"así":             "",
	"aun":             "",
	"aún":             "",
	"aunque":          "",
	"bajo":            "",
	"bastante":        "",
	"bien":            "",
	"breve":           "",
	"casi":            "",
	"cerca":           "",
	"claro":           "",
	"como":            "",
	"cómo":            "",
	"con":             "",
	"conmigo":         "",
	"contigo":         "",
	"contra":          "",
	"cual":            "",
	"cuál":            "",
	"cuales":          "",
	"cuáles":          "",
	"cuando":          "",
	"cuándo":          "",
	"cuanta":          "",
	"cuánta":          "",
	"cuantas":         "",
	"cuántas":         "",
	"cuanto":          "",
	"cuánto":          "",
	"cuantos":         "",
	"cuántos":         "",
	"de":              "",
	"debajo":          "",
	"del":             "",
	"delante":         "",
	"demasiado":       "",
	"dentro":          "",
	"deprisa":         "",
	"desde":           "",
	"despacio":        "",
	"despues":         "",
	"después":         "",
	"detras":          "",
	"detrás":          "",
	"dia":             "",
	"día":             "",
	"dias":            "",
	"días":            "",
	"donde":           "",
	"dónde":           "",
	"dos":             "",
	"durante":         "",
	"el":              "",
	"él":              "",
	"ella":            "",
	"ellas":           "",
	"ellos":           "",
	"en":              "",
	"encima":          "",
	"enfrente":        "",
	"enseguida":       "",
	"entre":           "",
	"es":              "",
	"esa":             "",
	"ésa":             "",
	"esas":            "",
	"ésas":            "",
	"ese":             "",
	"ése":             "",
	"eso":             "",
	"esos":            "",
	"ésos":            "",
	"esta":            "",
	"está":            "",
	"ésta":            "",
	"estado":          "",
	"estados":         "",
	"estan":           "",
	"están":           "",
	"estar":           "",
	"estas":           "",
	"éstas":           "",
	"este":            "",
	"éste":            "",
	"esto":            "",
	"estos":           "",
	"éstos":           "",
	"ex":              "",
	"excepto":         "",
	"final":           "",
	"fue":             "",
	"fuera":           "",
	"fueron":          "",
	"g":               "",
	"general":         "",
	"gran":            "",
	"ha":              "",
	"habia":           "",
	"había":           "",
	"habla":           "",
	"hablan":          "",
	"hace":            "",
	"hacia":           "",
	"han":             "",
	"hasta":           "",
	"hay":             "",
	"horas":           "",
	"hoy":             "",
	"i":               "",
	"incluso":         "",
	"informo":         "",
	"informó":         "",
	"junto":           "",
	"la":              "",
	"lado":            "",
	"las":             "",
	"le":              "",
	"lejos":           "",
	"lo":              "",
	"los":             "",
	"luego":           "",
	"mal":             "",
	"mas":             "",
	"más":             "",
	"mayor":           "",
	"me":              "",
	"medio":           "",
	"mejor":           "",
	"menos":           "",
	"menudo":          "",
	"mi":              "",
	"mí":              "",
	"mia":             "",
	"mía":             "",
	"mias":            "",
	"mías":            "",
	"mientras":        "",
	"mio":             "",
	"mío":             "",
	"mios":            "",
	"míos":            "",
	"mis":             "",
	"mismo":           "",
	"mucho":           "",
	"muy":             "",
	"nada":            "",
	"nadie":           "",
	"ninguna":         "",
	"no":              "",
	"nos":             "",
	"nosotras":        "",
	"nosotros":        "",
	"nuestra":         "",
	"nuestras":        "",
	"nuestro":         "",
	"nuestros":        "",
	"nueva":           "",
	"nuevo":           "",
	"nunca":           "",
	"os":              "",
	"otra":            "",
	"otros":           "",
	"pais":            "",
	"paìs":            "",
	"para":            "",
	"parte":           "",
	"pasado":          "",
	"peor":            "",
	"pero":            "",
	"poco":            "",
	"por":             "",
	"porque":          "",
	"pronto":          "",
	"proximo":         "",
	"próximo":         "",
	"puede":           "",
	"qeu":             "",
	"que":             "",
	"qué":             "",
	"quien":           "",
	"quién":           "",
	"quienes":         "",
	"quiénes":         "",
	"quiza":           "",
	"quizá":           "",
	"quizas":          "",
	"quizás":          "",
	"raras":           "",
	"repente":         "",
	"salvo":           "",
	"se":              "",
	"sé":              "",
	"segun":           "",
	"según":           "",
	"ser":             "",
	"sera":            "",
	"será":            "",
	"si":              "",
	"sí":              "",
	"sido":            "",
	"siempre":         "",
	"sin":             "",
	"sobre":           "",
	"solamente":       "",
	"solo":            "",
	"sólo":            "",
	"son":             "",
	"soyos":           "",
	"su":              "",
	"supuesto":        "",
	"sus":             "",
	"suya":            "",
	"suyas":           "",
	"suyo":            "",
	"tal":             "",
	"tambien":         "",
	"también":         "",
	"tampoco":         "",
	"tarde":           "",
	"te":              "",
	"temprano":        "",
	"ti":              "",
	"tiene":           "",
	"todavia":         "",
	"todavía":         "",
	"todo":            "",
	"todos":           "",
	"tras":            "",
	"tu":              "",
	"tú":              "",
	"tus":             "",
	"tuya":            "",
	"tuyas":           "",
	"tuyo":            "",
	"tuyos":           "",
	"un":              "",
	"una":             "",
	"unas":            "",
	"uno":             "",
	"unos":            "",
	"usted":           "",
	"ustedes":         "",
	"veces":           "",
	"vez":             "",
	"vosotras":        "",
	"vosotros":        "",
	"vuestra":         "",
	"vuestras":        "",
	"vuestro":         "",
	"vuestros":        "",
	"ya":              "",
	"yo":              ""}


In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

sinopsis = df_1['Sinopsis']

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words= spanish)
tfidf = tfidf_vectorizer.fit_transform(sinopsis)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=spanish)
tf = tf_vectorizer.fit_transform(sinopsis)
tf_feature_names = tf_vectorizer.get_feature_names()

In [81]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [82]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 10
print 'Modelo NMF:'
display_topics(nmf, tfidf_feature_names, no_top_words)
print '\nModelo LDA:'
display_topics(lda, tf_feature_names, no_top_words)

Modelo NMF:
Topic 0:
documental ficción argentina film guerra país acerca mujeres cuatro raúl
Topic 1:
vida joven mujer años familia padre vive más madre relación
Topic 2:
buenos aires ciudad calles semana literatura inmigrantes haber económica festival
Topic 3:
película cine cuenta diferentes acción basada diálogos cortometraje vida personajes
Topic 4:
historia cuenta narra amor llega locura viven trabajo amigos feliz
Topic 5:
dictadura militar última argentina desaparecido 1976 familia años gobierno federal
Topic 6:
href persona index personas php cinenacional http com www html
Topic 7:
pueblo pequeño año años patagonia regresa provincia habitantes vengarse solitario
Topic 8:
hombre trabaja trabajo mujer convierte decide realidad va jefe director
Topic 9:
br música luthiers espectáculo filmación les programa canción 1983 guión

Modelo LDA:
Topic 0:
documental br película argentina dictadura cine militar film acerca diez
Topic 1:
href persona www http cinenacional com aires buenos día

ajena/o/s
ajena
ajena/o/s
ajena/o
alg�n/a/o/s
alg�n
alg�n/a/o/s
alg�n/a
alg�n/a/o/s
alg�n/a/o
all�/�
all�
aquella/o/s
aquella
aquella/o/s
aquella/o
cierto/a/s
cierto
cierto/a/s
cierto/a
cualquier/a/s
cualquier
cualquier/a/s
cualquier/a
cuanto/a/s
cuanto
cuanto/a/s
cuanto/a
demasiada/o/s
demasiada
demasiada/o/s
demasiada/o
ella/o/s
ella
ella/o/s
ella/o
era/s
era
esa/e/o/s
esa
esa/e/o/s
esa/e
esa/e/o/s
esa/e/o
esta/s
esta
este/o/s
este
este/o/s
este/o
hace/s
hace
intenta/s
intenta
junto/s
junto
la/o/s
la
la/o/s
la/o
mi/s
mi
m�a/s
m�a
m�o/s
m�o
misma/o/s
misma
misma/o/s
misma/o
mucha/s
mucha
much�sima/o/s
much�sima
much�sima/o/s
much�sima/o
mucho/s
mucho
ning�n/a/o/s
ning�n
ning�n/a/o/s
ning�n/a
ning�n/a/o/s
ning�n/a/o
nosotras/os
nosotras
nuestra/o/s
nuestra
nuestra/o/s
nuestra/o
otra/o/s
otra
otra/o/s
otra/o
poca/o/s
poca
poca/o/s
poca/o
podr�a/s
podr�a
puede/n
puede
qui�n/es
qui�n
quiz�/s
quiz�
sabe/s/n
sabe
sabe/s/n
sabe/s
su/s
su
suya/o/s
suya
suya/o/s
suya/o
tal/es
tal
tanta/o/s
tan

{'a': '',
 'acuerdo': '',
 'adelante': '',
 'ademas': '',
 'adem\xc3\xa1s': '',
 'adrede': '',
 'ahi': '',
 'ahora': '',
 'ah\xc3\xad': '',
 'al': '',
 'alli': '',
 'all\xc3\xad': '',
 'alrededor': '',
 'antano': '',
 'anta\xc3\xb1o': '',
 'ante': '',
 'antes': '',
 'apenas': '',
 'aproximadamente': '',
 'aquel': '',
 'aquella': '',
 'aquellas': '',
 'aquello': '',
 'aquellos': '',
 'aqui': '',
 'aqu\xc3\xa9l': '',
 'aqu\xc3\xa9lla': '',
 'aqu\xc3\xa9llas': '',
 'aqu\xc3\xa9llos': '',
 'aqu\xc3\xad': '',
 'arribaabajo': '',
 'asi': '',
 'as\xc3\xad': '',
 'aun': '',
 'aunque': '',
 'a\xc3\xban': '',
 'bajo': '',
 'bastante': '',
 'bien': '',
 'breve': '',
 'casi': '',
 'cerca': '',
 'claro': '',
 'como': '',
 'con': '',
 'conmigo': '',
 'contigo': '',
 'contra': '',
 'cual': '',
 'cuales': '',
 'cuando': '',
 'cuanta': '',
 'cuantas': '',
 'cuanto': '',
 'cuantos': '',
 'cu\xc3\xa1l': '',
 'cu\xc3\xa1les': '',
 'cu\xc3\xa1ndo': '',
 'cu\xc3\xa1nta': '',
 'cu\xc3\xa1ntas': '',
 'cu\xc3\